# 6. users.json

Requirements

In [1]:
%pip install -r ../requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


Imports

In [2]:
import os
import time

import pandas as pd
import numpy as np

import json
from tqdm import tqdm
import logging

import uuid
import random
from datetime import datetime, timedelta

Logging configuration

In [3]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger()

Constants

In [4]:
# Repositories
DATA_DIR = os.path.join("..", "data")
TEMP_DIR = os.path.join(DATA_DIR, "temp")
RAW_DIR = os.path.join(DATA_DIR, "raw")
MONGO_DIR = os.path.join(DATA_DIR, "clean", "mongo")
NEO4J_DIR = os.path.join(DATA_DIR, "clean", "neo4j")

In [5]:
# Repos creation
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(TEMP_DIR, exist_ok=True)
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(MONGO_DIR, exist_ok=True)
os.makedirs(NEO4J_DIR, exist_ok=True)

In [6]:
# Files
NEO4J_FOLLOWERS = os.path.join(NEO4J_DIR, "followers.json")
NEO4J_WINNERS_JSON = os.path.join(NEO4J_DIR, "winners.json")
NEO4J_PARTICIPANTS_JSON = os.path.join(NEO4J_DIR, "participants.json")
NEO4J_ADMINISTRATORS_JSON = os.path.join(NEO4J_DIR, "administrators.json")

MONGO_USERS_JSON = os.path.join(MONGO_DIR, "users.json")

Utility functions

In [7]:
def save_to_json(data, filename):
    """
    Save data to a JSON file.

    Parameters:
    data (any): The data to be saved to the JSON file. This can be any data type that is serializable to JSON.
    filename (str): The name of the file where the data will be saved.

    Returns:
    None
    """
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

## Execution

In [8]:
with open(NEO4J_FOLLOWERS, "r", encoding="utf-8") as f:
    relationships = json.load(f)
logger.info(f"Followers: {len(relationships)}")

with open(NEO4J_WINNERS_JSON, "r", encoding="utf-8") as f:
    winners = json.load(f)
logger.info(f"Winners: {len(winners)}")

with open(NEO4J_PARTICIPANTS_JSON, "r", encoding="utf-8") as f:
    participants = json.load(f)
logger.info(f"Participants: {len(participants)}")

with open(NEO4J_ADMINISTRATORS_JSON, "r", encoding="utf-8") as f:
    administrators = json.load(f)
logger.info(f"Administrators: {len(administrators)}")

with open(MONGO_USERS_JSON, "r", encoding="utf-8") as f:
    users = json.load(f)
logger.info(f"Users: {len(users)}")

2025-01-21 11:25:40,111 - INFO - Followers: 259629
2025-01-21 11:25:40,134 - INFO - Winners: 9799
2025-01-21 11:25:41,184 - INFO - Participants: 531808
2025-01-21 11:25:41,235 - INFO - Administrators: 20006
2025-01-21 11:25:41,485 - INFO - Users: 5000


In [16]:
final_users = []
for user in tqdm(users, desc="Users"):

    # Set computed data
    user["followers"] = len([x for x in relationships if x["followed"] == user["username"]])
    user["following"] = len([x for x in relationships if x["follower"] == user["username"]])
    user["tournaments"] = {
        "partecipated": len([x for x in participants if x["participant"] == user["username"]]),
        "won": len([x for x in winners if x["winner"] == user["id"]]),
        "created": len([x for x in administrators if x["administrator"] == user["username"]])
    }

    final_users.append(user)

Users:   0%|          | 0/5000 [00:00<?, ?it/s]

Users: 100%|██████████| 5000/5000 [13:10<00:00,  6.32it/s]


In [17]:
[user for user in final_users if user["username"] == "respino"]

[{'id': '40acdbcf-be6d-4e9c-a6bc-a23e3515d3fd',
  'username': 'respino',
  'firstName': 'Emanuele',
  'lastName': 'Respino',
  'email': 'emanuele.respino4517@yahoo.com',
  'password': '87b555c0457fb41ebe56d877761376315014441693aece27297664aa02541aeb',
  'birthDate': '1956-05-01',
  'location': {'city': None, 'stateOrProvince': 'Toscana', 'country': 'Italy'},
  'followers': 9,
  'following': 40,
  'tournaments': {'partecipated': 20, 'won': 0, 'created': 4},
  'mostRecentReviews': [{'id': 'bed14528-79a1-44f1-950c-637b1950cbb0',
    'postDate': '2022-11-25T00:00:00Z',
    'game': {'id': '728b498f-bffd-40f0-9013-8dfcb434571b',
     'name': 'Oathsworn: Into the Deepwood',
     'yearReleased': 2022},
    'rating': 7.0,
    'content': 'Play with a friend who owns a copy. Brilliant game, terrific story and atmosphere, intense battles.'},
   {'id': '63a3a9ed-d61c-4e86-a5c2-6111d405b9cf',
    'postDate': '2022-09-14T00:00:00Z',
    'game': {'id': '5f0df14a-8760-47b8-8130-8da8591af0c9',
     'nam

In [18]:
with open(MONGO_USERS_JSON, "w", encoding="utf-8") as f:
    json.dump(final_users, f, ensure_ascii=False, indent=4)
logger.info(f"Users: {len(final_users)}")

2025-01-21 11:52:25,699 - INFO - Users: 5000
